In [1]:
import torch
from torch import nn
from d2l import torch as d2l


In [4]:
def corr2d(X,K):
    h,w = K.shape
    print(K.shape)
    Y = torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i][j] = (X[i:i+h,j:j+w]*K).sum()
    return Y



In [6]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0]])
corr2d(X, K)

torch.Size([1, 2])


tensor([[1., 2.],
        [4., 5.],
        [7., 8.]])

In [7]:
# 定义卷积层
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super.__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self,x):
        return corr2d(x,self.weight)+self.bias
    

In [8]:
# 简单的边缘检测
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [9]:
K = torch.tensor([[1.0, -1.0]])

In [10]:
Y = corr2d(X, K)
Y

torch.Size([1, 2])


tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [12]:
# 学习卷积核
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6-1+1,8-2+1))
lr = 3e-2

for i in range(20):
    Y_hat = conv2d(X)
    l = (Y_hat-Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')


epoch 2, loss 15.097
epoch 4, loss 4.761
epoch 6, loss 1.711
epoch 8, loss 0.661
epoch 10, loss 0.264
epoch 12, loss 0.107
epoch 14, loss 0.044
epoch 16, loss 0.018
epoch 18, loss 0.007
epoch 20, loss 0.003


In [13]:
conv2d.weight.data

tensor([[[[ 0.9942, -1.0055]]]])